In [ ]:
from google.colab import files

# Upload file manually
uploaded = files.upload()



Saving data.csv to data.csv


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.cluster import KMeans

# Load dataset
df = pd.read_csv("data.csv")

# Step 1: **Group Stations by Line Instead of Exact Names**
df["station_group"] = df["lineNameEng"]  # Use train line names as target labels

# Encode station group labels
label_encoder = LabelEncoder()
df["station_encoded"] = label_encoder.fit_transform(df["station_group"])

# Normalize Latitude & Longitude
scaler = StandardScaler()
df[["geoLat", "geoLng"]] = scaler.fit_transform(df[["geoLat", "geoLng"]])

# Define Input (X) and Output (y)
X = df[["geoLat", "geoLng"]].values
y = df["station_encoded"].values  # Now predicting train lines, not exact stations

# Step 2: **Apply K-Means Clustering to Simplify Similar Stations**
kmeans = KMeans(n_clusters=15, random_state=42)  # 15 station clusters
df["station_cluster"] = kmeans.fit_predict(X)

# Step 3: **Balance Data Using Oversampling**
oversample = RandomOverSampler(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = oversample.fit_resample(X, y)

# Split Train/Test Data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 4: **Build an Improved Neural Network**
model = Sequential([
    Dense(256, input_shape=(2,)),
    LeakyReLU(alpha=0.1),  # Use LeakyReLU instead of ReLU
    BatchNormalization(),
    Dropout(0.4),

    Dense(128),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),

    Dense(len(label_encoder.classes_), activation="softmax")  # Multi-class classification
])

# Compile Model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train Model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Fixed Neural Network Accuracy: {accuracy:.2%}")



Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.2003 - loss: 2.4035 - val_accuracy: 0.4462 - val_loss: 1.8386
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4576 - loss: 1.4418 - val_accuracy: 0.4769 - val_loss: 1.7823
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5069 - loss: 1.2390 - val_accuracy: 0.4462 - val_loss: 1.7502
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5777 - loss: 1.1211 - val_accuracy: 0.4462 - val_loss: 1.7293
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6591 - loss: 0.9770 - val_accuracy: 0.4462 - val_loss: 1.7145
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6610 - loss: 1.0822 - val_accuracy: 0.4000 - val_loss: 1.7007
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6684 - loss: 0.9050 - val_accuracy: 0.3231 - val_loss: 1.6818
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6845 - loss: 0.9044 - val_accuracy: 0.3231 - val_loss: 1.6651
Epoc

✅ Final Fixed Neural Network Accuracy: 76.92%


In [ ]:
import joblib
import tensorflow as tf
from google.colab import files

# Save model in the latest Keras format
model.save("bangkok_train_nn_final.keras")

# Save encoders and scalers
joblib.dump(label_encoder, "station_label_encoder_final.pkl")
joblib.dump(scaler, "lat_lng_scaler_final.pkl")

# Download the files to your local machine
files.download("bangkok_train_nn_final.keras")
files.download("station_label_encoder_final.pkl")
files.download("lat_lng_scaler_final.pkl")

print("✅ Model and required files saved & ready for download.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Model and required files saved & ready for download.
